In [1]:
import matplotlib.pyplot as plt
import numpy as np
import librosa
import IPython
import torchaudio
import numpy as np
import soundfile as sf
import torchaudio.functional as F
import torch
import os
import torchaudio
import os

In [2]:
# audio_len = 16000 * 5
# audio_len = None

In [2]:
sample_rate = 16000

speech0, sr = torchaudio.load(
    "/data/ephraim/datasets/known_noise/exp2/clean_wav/ffbb695d_nohash_0_0.wav"
)
speech1, sr = torchaudio.load(
    "/data/ephraim/datasets/known_noise/exp2/clean_wav/ffbb695d_nohash_0_9.wav"
)
# if sr != sample_rate:
#     speech = F.resample(speech, sr, sample_rate)

In [4]:
speech = torch.concat((speech0, speech1), dim=1)
# speech = speech[:, :audio_len]
# speech = torch.clamp(speech[0], -1.0, 1.0)


# add gaussian noise
# noisy_wav = wav + np.random.normal(mean_, std_, wav.shape)
noise = torch.randn(1, speech.shape[1])
speech.shape

torch.Size([1, 32000])

In [5]:
noise.shape

torch.Size([1, 32000])

In [6]:
for snr in [-10,-5,0,5,-15,-20]:
    # snr = 10
    noise_power = 1 / noise.shape[1] * torch.sum(noise**2)
    speech_power = 1 / speech.shape[1] * torch.sum(speech**2)
    noise_power_target = speech_power * np.power(10, -snr / 10)
    noise_scaling = torch.sqrt(noise_power_target / noise_power)
    lossy_speech = speech + noise_scaling * noise
    
    noise_var = noise_scaling**2
    
    new_noise = noise_scaling * noise
    new_noise_power = 1 / new_noise.shape[1] * torch.sum(new_noise**2)
    
    if torch.max(lossy_speech) >= 1 or torch.min(lossy_speech) <= -1:
        print(snr, " attention!")
        
    filename = "digits_snr{}_var{}.wav".format(snr, noise_var)

    tarpath = os.path.join("/data/ephraim/datasets/known_noise/exp2/clean_wav/", filename)
    torchaudio.save(tarpath, speech, sample_rate)

    tar_noisypath = os.path.join("/data/ephraim/datasets/known_noise/exp2/noisy_wav/", filename)
    torchaudio.save(tar_noisypath, lossy_speech, sample_rate)
    

-15  attention!
-20  attention!


In [26]:
noise_var = noise_scaling**2
noise_var
torch.std(speech)**2

tensor(0.0005)

In [39]:
new_noise = noise_scaling * noise
new_noise_power = 1 / new_noise.shape[1] * torch.sum(new_noise**2)
new_noise_power

tensor(0.0005)

In [28]:
if torch.max(lossy_speech) >= 1 or torch.min(lossy_speech) <= -1:
    print("attention!")

In [6]:
from IPython.display import Audio, display

display(Audio(lossy_speech, rate=sample_rate, autoplay=False))

In [30]:
display(Audio(speech, rate=sample_rate, autoplay=False))

In [31]:
filename = "digits_snr{}_var{}.wav".format(snr, noise_var)
filename

'digits_snr10_var0.00047484078095294535.wav'

In [15]:
# sf.write(
#     os.path.join("/data/ephraim/datasets/known_noise/clean_wav/", filename),
#     speech,
#     sr,
# )

In [32]:
import torchaudio
import os

tarpath = os.path.join("/data/ephraim/datasets/known_noise/clean_wav/", filename)
torchaudio.save(tarpath, speech, sample_rate)

tar_noisypath = os.path.join("/data/ephraim/datasets/known_noise/noisy_wav/", filename)
torchaudio.save(tar_noisypath, lossy_speech, sample_rate)

In [26]:
display(Audio(speech, rate=sample_rate, autoplay=False))

In [27]:
torch.std(speech)

tensor(0.0106)